In [0]:
import csv
import pandas as pd
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout, ThresholdedReLU, Flatten
from keras.models import Sequential
from tensorflow.keras import Model
tf.test.gpu_device_name() 

'/device:GPU:0'

In [0]:
# Code to read csv file into Colaboratory:
from google.colab import drive
drive.mount('/content/drive')
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# downloaded = drive.CreateFile({'id':'1LAXYuJapp6u6shheA3ciIy67GrOQdlKs'}) 
# downloaded.GetContentFile('q2_revised.csv')  
df2 = pd.read_csv('/content/drive/Shared drives/Hackathan/q2_revised.csv', encoding='latin1')
# Dataset is now stored in a Pandas Dataframe
gt2 = pd.read_excel('/content/drive/Shared drives/Hackathan/q2_answer.xlsx')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: ignored

In [0]:
df2.head()

,No.,Time,Source,Destination,Protocol,Length,Text item,Transaction ID,Name,Time.1,Reply code,Response,Primary name server,Info,dns
0,1,0.000000,183.213.21.48,166.111.8.29,DNS,73,â\234\223,0x2719,weilaigou.com,NaN,NaN,Message is a query,NaN,Standard query 0x2719 A weilaigou.com,weilaigou.com
1,2,0.044210,8.8.8.8,101.6.69.132,DNS,102,â\234\223,0x51ae,taobao.com,NaN,No error,Message is a response,NaN,Standard query response 0x51ae A taobao.com A ...,taobao.com
2,3,0.069832,8.8.8.8,101.6.128.60,DNS,68,â\234\223,0xf102,sohu.com,NaN,Server failure,Message is a response,NaN,Standard query response 0xf102 Server failure ...,sohu.com
3,4,0.090890,144.217.10.132,166.111.124.17,DNS,79,â\234\223,0xbde3,hrsa.gov,NaN,NaN,Message is a query,NaN,Standard query 0xbde3 ANY hrsa.gov OPT,hrsa.gov
4,5,0.147890,183.173.94.53,8.8.8.8,DNS,70,â\234\223,0xb6ce,wahom.site,NaN,NaN,Message is a query,NaN,Standard query 0xb6ce A wahom.site,wahom.site


In [0]:
def extractDNS(info):
    info = info.split(' ')
    for s in info:
        if '.' in s:
            return s
    
    return ''

In [0]:
df2['dns'] = df2.Info.apply(extractDNS)

# Part 1: classify the malicious DNS and benign DNS

### prepare the data
input is the DNS string
output is 1 or 0. 1 means malicious and 0 means benign

In [0]:
data_x = df2.dns.unique()

In [0]:
# check all the ground truth dns are in the train_x
x_set = set(data_x)
for dns in gt2.domain:
    if dns not in x_set:
        print(dns)

In [0]:
gt_set = set()
repeat_dns = []
for dns in gt2.domain:
    if dns not in gt_set:
        gt_set.add(dns)
    else:
        repeat_dns.append(dns)

In [0]:
drop_idx = [] #drop index
for dns in repeat_dns:
    drop_idx.append(gt2[(gt2.domain==dns) & (gt2.family == 1)].index[0])

ngt2 = gt2.drop(drop_idx) #new answers
ngt2

In [0]:
gt_set = set(ngt2.domain)
data_y = []
for dns in data_x:
    if dns in gt_set:
        idx = ngt2.index[ngt2['domain'] == dns].tolist()[0] #get index where domain = dns
        family = ngt2.at[int(idx), 'family'] #get family value from index
        data_y.append(family)
    else:
        data_y.append(0) #0 if not DGA
data_y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [0]:
p1_data = pd.DataFrame(list(zip(data_x, data_y)), columns =['dns', 'family'])
p1_data = p1_data.sample(frac=1, random_state=2020)
p1_data

,dns,family
47677,scylgroup.com,0
48162,effilnuds.com,0
28840,lovebookonline.com,0
4204,wwww91k.com,0
30276,93c53.com,0
...,...,...
18523,sousoucili.com,0
44739,ngmc.com,0
47734,weborama.fr,0
40259,mg4150.com,0


In [0]:
p1_data['dnsLength'] = p1_data.dns.apply(lambda x: len(x))

In [0]:
def removeEnding(domain):
  domain = domain.split(".", 1)[0]
  return domain

In [0]:
p1_data['dns'] = p1_data.dns.apply(lambda x: removeEnding(x))
p1

In [0]:
def dnsToMatrix(dns):
    maxl = 63
#     demision = 128
    asc = [ord(i.lower()) for i in dns]
    asc = [0] * (maxl-len(asc)) + asc
    asc = np.array(asc)
#     res = np.zeros((demision, maxl))
#     res[asc, np.arange(asc.size),] = 1
    
    return asc

In [0]:
np_x = []
for dns in p1_data.dns:
    np_x.append(dnsToMatrix(dns))
np_x = np.stack(np_x, axis=0)

In [0]:
np_x.shape
np_x

array([[  0,   0,   0, ...,  99, 111, 109],
       [  0,   0,   0, ...,  99, 111, 109],
       [  0,   0,   0, ...,  99, 111, 109],
       ...,
       [  0,   0,   0, ...,  46, 102, 114],
       [  0,   0,   0, ...,  99, 111, 109],
       [  0,   0,   0, ...,  46, 114, 117]])

In [0]:
def familyToMatrix(family): #turns 8 into [0,0,0,0,0,0,0,0,1,0,0] index 8 becomes 1
  out = [0]*11
  out[family] = 1
  return np.array(out)

In [0]:
np_y = []
for family in p1_data.family:
  np_y.append(familyToMatrix(family)) #1 hot encoding
np_y = np.stack(np_y, axis=0)
print(np_y.shape)
np_y

(56625, 11)


array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
n = len(p1_data)
train_ratio = 0.8 
val_ratio = 0.1
test_ratio = 0.1

train_x = np_x[:int(n*train_ratio),:]
train_y = np_y[:int(n*train_ratio),:]

val_x = np_x[int(n*train_ratio):int(n*(train_ratio+val_ratio)),:]
val_y = np_y[int(n*train_ratio):int(n*(train_ratio+val_ratio)),:]

test_x = np_x[int(n*(train_ratio+val_ratio)):,:]
test_y = np_y[int(n*(train_ratio+val_ratio)):,:]


DGA ratio in train: 1.1810154525386314
DGA ratio in val: 1.1898622394913458
DGA ratio in test: 1.1670492671728765


In [0]:
test_x.shape
#test_y.shape

(5663, 63)

###MIT MODEL

In [0]:
main_input = Input(shape=(63,), dtype='int32', name='input')
embedding = Embedding(input_dim=128, output_dim=128, input_length=63)(main_input)
conv = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu', strides=1)(embedding)
max_pool = MaxPooling1D(pool_size=2, padding='same')(conv) 
encode = LSTM(64, return_sequences=False)(max_pool)
output = Dense(11, activation='softmax')(encode) #11 outputs

model = Model(inputs=main_input, outputs=output)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 63)]              0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 63, 128)           16384     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 63, 128)           65664     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 32, 128)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_7 (Dense)              (None, 11)                715       
Total params: 132,171
Trainable params: 132,171
Non-trainable params: 0
_____________________________________________________

In [0]:
history = model.fit(train_x, train_y,
                    batch_size=64,
                    epochs=10,
                    validation_data=(val_x, val_y))

Epoch 1/10
708/708 [==============================] - 13s 18ms/step - loss: 0.2847 - accuracy: 0.8954 - val_loss: 0.1631 - val_accuracy: 0.9214
Epoch 2/10
708/708 [==============================] - 13s 18ms/step - loss: 0.1469 - accuracy: 0.9302 - val_loss: 0.1455 - val_accuracy: 0.9283
Epoch 3/10
708/708 [==============================] - 13s 18ms/step - loss: 0.1360 - accuracy: 0.9342 - val_loss: 0.1392 - val_accuracy: 0.9324
Epoch 4/10
708/708 [==============================] - 13s 18ms/step - loss: 0.1322 - accuracy: 0.9360 - val_loss: 0.1343 - val_accuracy: 0.9362
Epoch 5/10
708/708 [==============================] - 12s 18ms/step - loss: 0.1281 - accuracy: 0.9379 - val_loss: 0.1588 - val_accuracy: 0.9207
Epoch 6/10
708/708 [==============================] - 13s 18ms/step - loss: 0.1262 - accuracy: 0.9394 - val_loss: 0.1380 - val_accuracy: 0.9368
Epoch 7/10
708/708 [==============================] - 13s 18ms/step - loss: 0.1246 - accuracy: 0.9400 - val_loss: 0.1331 - val_accuracy:

In [0]:
test_loss, test_accuracy = model.evaluate(test_x, test_y)
print(test_accuracy)

177/177 [==============================] - 1s 7ms/step - loss: 0.1301 - accuracy: 0.9373
0.9373123645782471


In [0]:
y = model.predict(test_x)

In [0]:
threshold = 0.9

# End Game Model

In [0]:
main_input = Input(shape=(63, ), dtype='int32', name='main_input')
embedding = Embedding(input_dim=128, output_dim=128, input_length=63)(main_input)
lstm = LSTM(128, return_sequences=False)(embedding)
drop = Dropout(0.5)(lstm)
output = Dense(11, activation='softmax')(drop)

EndgameModel = Model(inputs=main_input, outputs=output)
EndgameModel.summary()
EndgameModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])


Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 63)]              0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 63, 128)           16384     
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 11)                1419      
Total params: 149,387
Trainable params: 149,387
Non-trainable params: 0
_________________________________________________________________


In [0]:
historyEndgame = EndgameModel.fit(train_x, train_y,
                    batch_size=64,
                    epochs=10,
                    validation_data=(val_x, val_y))

Epoch 1/10
708/708 [==============================] - 17s 24ms/step - loss: 0.3565 - val_loss: 0.1879
Epoch 2/10
708/708 [==============================] - 16s 23ms/step - loss: 0.1826 - val_loss: 0.1669
Epoch 3/10
708/708 [==============================] - 16s 23ms/step - loss: 0.1580 - val_loss: 0.1521
Epoch 4/10
708/708 [==============================] - 17s 23ms/step - loss: 0.1465 - val_loss: 0.1380
Epoch 5/10
708/708 [==============================] - 17s 24ms/step - loss: 0.1393 - val_loss: 0.1378
Epoch 6/10
708/708 [==============================] - 17s 24ms/step - loss: 0.1367 - val_loss: 0.1327
Epoch 7/10
708/708 [==============================] - 17s 23ms/step - loss: 0.1349 - val_loss: 0.1318
Epoch 8/10
708/708 [==============================] - 16s 23ms/step - loss: 0.1320 - val_loss: 0.1325
Epoch 9/10
708/708 [==============================] - 17s 23ms/step - loss: 0.1310 - val_loss: 0.1385
Epoch 10/10
708/708 [==============================] - 17s 24ms/step - loss: 0.129

In [0]:
test_loss, test_accuracy = model.evaluate(test_x, test_y)
print(test_accuracy)

177/177 [==============================] - 1s 7ms/step - loss: 0.1301 - accuracy: 0.9373
0.9373123645782471
